In [ ]:
!pip install openai==0.28

In [ ]:
import pandas as pd
import openai

In [ ]:
trainA = pd.read_csv('TaskA-TrainingSet.csv')

In [ ]:
sections = trainA['section_header'].unique().tolist()
max_tokens = {}

for s in sections:
  s_df = trainA[trainA['section_header'] == s]
  avg_chars = s_df['section_text'].str.len().mean()
  max_tokens[s] = int(round(avg_chars/4))

In [ ]:
abb_to_full = {
 'GENHX': "History of Present Illness",
 'ROS': "Review of Systems",
 'PASTMEDICALHX': "Past Medical History",
 'MEDICATIONS': "Medications",
 'CC': "Chief Complaint",
 'PASTSURGICAL': "Past Surgical History",
 'FAM/SOCHX': "Family History/Social History",
 'DISPOSITION': "Disposition",
 'DIAGNOSIS': "Diagnosis",
 'EDCOURSE': "Emergency Department Course",
 'PLAN': "Plan",
 'LABS': "Labs",
 'ASSESSMENT': "Assessment",
 'ALLERGY': "Allergy",
 'GYNHX': "Gynecologic History",
 'EXAM': "Exam",
 'OTHER_HISTORY': "Other History",
 'PROCEDURES': "Procedures",
 'IMAGING': "Imaging",
 'IMMUNIZATIONS': "Immunizations"
}

In [ ]:
prompt_per_section = pd.read_csv("best_prompt_per_section.csv")
prompt_per_section_dic = prompt_per_section.set_index('Section')['Best_Prompt'].to_dict()

In [ ]:
openai.api_key = #insert API key here

In [ ]:
def get_specific_prompt(section, dia):
  prompt_template = prompt_per_section_dic[section]
  cleaned_prompt = prompt_template[2:-1]+"\n"
  prompt = eval(f'f"""{cleaned_prompt}"""')
  return prompt

In [ ]:
def get_generic_prompt(section, dia):
  section = abb_to_full[section]
  generic_prompt = """Generate a summary for the section "{section}" in a SOAP\
  note based on the provided doctor-patient dialogue.\n\nDialogue:\n{dia}\n\n{section}\
  Summary:""".format(section=section, dia=dia)
  return generic_prompt

In [ ]:
def get_pred(model, prompt, max_tokens):
  response = openai.ChatCompletion.create(
              model=model,
              messages=[
                  {"role": "system", "content": "You are a smart doctor as well as a professional medical scribe with a lot of experience."},
                  {"role": "user", "content": prompt},
              ],
              temperature=0.2,
              max_tokens=max_tokens,
              timeout=45,
              n=1,
              stop=None
          )

  return response['choices'][0]['message']['content']

In [ ]:
model = #"gpt-3.5-turbo" # "gpt-4" #use with the model you want

In [ ]:
trainA[f'{model}_specific'] = trainA.apply(lambda row: get_pred(model, get_specific_prompt(row['section_header'], row['dialogue']), max_tokens[row['section_header']]), axis=1)

In [ ]:
trainA[f'{model}_generic'] = trainA.apply(lambda row: get_pred(model, get_generic_prompt(row['section_header'], row['dialogue']), max_tokens[row['section_header']]), axis=1)

In [ ]:
trainA.to_csv("specific_generic_all_data.csv")